In [1]:
import torch
import torch.nn as nn
from codebase.model import Model
from codebase.data import Dataset
from codebase.train import train, step
from codebase.inference import generate, tokens_to_segs
from codebase.utils import load_pkl, plot_list
from codebase.preprocessing import create_dataset

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using device: {device}')

Using device: cuda


In [3]:
create_dataset(
    nocturnes=True,
    track_idx=0,
    dataset_path='maestro-v3.0.0',
    output_file='small.pkl',
    seg_fit_tightness=0.12
)

Loading paths...
Found 1 tracks
Processing track 1/1: MIDI-Unprocessed_R1_D1-9-12_mid--AUDIO-from_mp3_12_R1_2015_wav--3.midi
Saving dataset to small.pkl...
Dataset creation complete!


In [2]:
dataset = load_pkl('small.pkl')

In [3]:
model = Model(
    d_model= 216,
    nhead=4,
    num_encoder_layers=3,
    num_decoder_layers=3,
    dim_feedforward=800,
    dropout=0.1
).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [3]:
model = Model.load("saves/model.pt").to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [4]:
train('train_dataset.pkl', 32, 1e-3, 200, device, model)

Using provided model
Step 100/200, Avg Loss: -13.8645
Step 200/200, Avg Loss: -13.3764
Saving model to saves/model.pt


In [12]:
notes, tokens = dataset(4, 150)
generated_tokens = generate(model, notes, max_length=127, device=device)
generated_segs = tokens_to_segs(generated_tokens)
ground_truth_segs = tokens_to_segs(tokens)
fig = plot_list(ground_truth_segs, generated_segs, title='Test Generation', tokens=generated_tokens)
fig.show()
print(f"{len(generated_tokens)} tokens generated")
print(f"{len(tokens)} gt tokens")

4 tokens generated
29 gt tokens
